In [1]:
import numpy as np
import torch
import sys

sys.path.append('..')
from utils import load_data

# datasets = [
#     'Flickr',
#     'ACM',
#     'BlogCatalog',
#     'Cora',
#     'Citeseer',
#     'Pubmed',
#     'CoraFull',
#     'arxiv',
#     'products',
#     'Reddit',
# ]

# !!! ACM 还有孤立节点
# !!! BlogCatalog, Flickr 的 label 从 1 开始
datasets = ['Cora']
for ds in datasets:
    graph, label, n_lbs = load_data(ds)
    features = graph.ndata["feat"]
    edges = graph.edges()
    adj = graph.adj()
    n_nodes = graph.num_nodes()
    u, v = edges[0].numpy(), edges[1].numpy()

    label_np_plus1 = label.numpy() + 1
    st = 2 if ds in ['BlogCatalog', 'Flickr'] else 1
    label_dicts = {l: np.nonzero(label_np_plus1 == l)[0] for l in np.unique(label_np_plus1)}
    mask = np.repeat([label_np_plus1], n_nodes, axis=0)
    




  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.




In [2]:
label_adj = adj.to_dense().numpy() * mask
label_nei = np.zeros((n_nodes, n_lbs))

print(st, n_lbs + st)
# 统计各节点的邻域节点的类别计数
for l in range(st, n_lbs + st):
    label_nei[:,l - st] = (label_adj == l).sum(1)

# 验证所有节点都不是孤立节点
np.all(label_nei.sum(1, keepdims=True) > 0)

# 计数转化为占比
label_nei_d = label_nei / label_nei.sum(1, keepdims=True)

1 8


In [3]:
# 整图所有节点的邻域节点类别分布情况
# 最大占比，最小占比，整图邻域类别分布方差
print(label_nei_d.mean(0).max(),'\t',label_nei_d.mean(0).min(),'\t',  ((label_nei_d - label_nei_d.mean(0))**2).mean())

0.29259344287704914 	 0.06432331963885082 	 0.09770126920937716


In [4]:
# 各类类内节点邻域类别分布情况
label_mean = 0
for i in range(st, n_lbs+st):
    tt = label_nei_d[label_dicts[i]]
    label_mean = label_mean + ((tt - tt.mean(0))**2).mean()
    print(tt.shape[0], '\t', tt.mean(0).max(),'\t',tt.mean(0).min(),'\t', ((tt - tt.mean(0))**2).mean())
label_mean / n_lbs

351 	 0.7433252651965693 	 0.013672957677482563 	 0.03051319510441177
217 	 0.7686691036667316 	 0.0015360983102918587 	 0.031574972901707556
418 	 0.916962929241898 	 0.0005980861244019139 	 0.010894852248660967
818 	 0.8385238729917665 	 0.003760138937400552 	 0.0244429087405303
426 	 0.848841728517578 	 0.0016431924882629107 	 0.024134556330106048
298 	 0.7855714732418337 	 0.016996095265439912 	 0.0291834362871841
180 	 0.788384031314846 	 0.0009096816114359974 	 0.023799144488337646


0.02493472372870548

/data/guming/anaconda3/envs/ecd/lib/python3.6/site-packages/outdated/utils.py:18: OutdatedCacheFailedWarning: Failed to use cache while checking for outdated package.
Set the environment variable OUTDATED_RAISE_EXCEPTION=1 for a full traceback.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs


结论：<font color=red><strong>各类类内邻域类别分布方差 < 整图邻域类别分布方差</strong></font>